In [ ]:
!pip install pymupdf python-docx azure-ai-formrecognizer


In [ ]:
!git clone https://github.com/williamli-15/pdf2layout2docx.git
%cd pdf2layout2docx

**Create a folder named docs in the project directory**

In [26]:
!mkdir -p docs

**and place your PDF files in it.**

In [ ]:
import os

# Set Azure Form Recognizer endpoint URL and API key
os.environ["AZ_ENDPOINT"] = "https://your-cognitiveservices-endpoint-url/"
os.environ["AZ_KEY"] = "your-form-recognizer-api-key"

In [ ]:
# Obtain JSON files from PDFs
!python obtain_json.py ./docs

In [17]:
# Process JSON files
!python processing_json.py ./docs

/content/pdf2layout2docx/docs/CA Infrastructure Fin.pdf
done!


Convert a single PDF file, replacing file_name with the base name of your PDF file (without the extension)

Using 0 and 1 as the start_page and end_page arguments would convert only the first page.

Using 0 and 2 would convert the first and second pages.

In [ ]:
file_name = 'CA Infrastructure Fin'  # Replace this with the base name of your PDF file
pdf_path = f'./docs/{file_name}.pdf'
out_path = f'./docs/{file_name}.docx'
json_path = f'./docs/updated_{file_name}.json'
start_page = 0
end_page = 2

!python convert_main.py $pdf_path $out_path $json_path $start_page $end_page


Convert all PDF files in the docs folder

In [ ]:
import os
import json

# Define the path to the docs folder
docs_folder = "./docs"

# Function to get PDF and JSON file pairs in the specified directory
def get_file_pairs(directory):
    file_pairs = []
    for file_name in os.listdir(directory):
        if file_name.endswith(".pdf"):
            pdf_path = os.path.join(directory, file_name)
            json_name = f"updated_{os.path.splitext(file_name)[0]}.json"
            json_path = os.path.join(directory, json_name)
            if os.path.exists(json_path):  # Check if corresponding JSON file exists
                file_pairs.append((pdf_path, json_path))
    return file_pairs

# Function to extract end page number from JSON
def extract_end_page_number(json_path):
    with open(json_path, "r") as file:
        data = json.load(file)
    end_page_number = max(page["page_number"] for page in data["pages"])
    return end_page_number

# Get PDF and JSON file pairs in the docs folder
file_pairs = get_file_pairs(docs_folder)

# Iterate over the file pairs and perform conversion
for pdf_path, json_path in file_pairs:
    base_name = os.path.splitext(os.path.basename(pdf_path))[0]
    out_path = os.path.join(docs_folder, f"{base_name}.docx")

    # Extract the end page number
    end_page = extract_end_page_number(json_path)

    # Set the start page to 0 (assuming it always starts from the first page)
    start_page = 0

    # Run the conversion script with the specified parameters
    !python convert_main.py $pdf_path $out_path $json_path $start_page $end_page
